In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from pymongo import MongoClient
import time
import locale

In [2]:
def login():
    elem = driver.find_element_by_id('mailbox:saveauth')
    elem.send_keys(Keys.SPACE)
    elem = driver.find_element_by_id('mailbox:login')
    elem.send_keys('study.ai_172')
    elem.send_keys(Keys.RETURN)
    elem = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.ID, 'mailbox:password')))
    elem.send_keys('NewPassword172')
    elem.send_keys(Keys.RETURN)    

In [3]:
def get_date(elem):
    elem = elem.find_element_by_class_name('letter__date').text
    try:
        if 'Вчера' in elem:
            date = datetime.strptime(elem.split(' ')[1] + str(datetime.date(datetime.now()) - timedelta(days=1)), '%H:%M%Y-%m-%d')
        elif 'Сегодня' in elem:
            date = datetime.strptime(elem.split(' ')[1] + str(datetime.date(datetime.now())), '%H:%M%Y-%m-%d')
        else:
            date = datetime.strptime(elem + str(datetime.now().year), '%d %B, %H:%M%Y')
    except:
        date = None
    return date

In [4]:
def get_text(elem):
    try:
        text = elem.find_element_by_xpath('//*[contains(@id,"mailru_css_attribute_postfix")]').text
    except:
        try:
            text = elem.find_element_by_xpath('//*[contains(@class,"mailru_css_attribute_postfix")]').text
        except:
            text = None
    return text

In [5]:
locale.setlocale(locale.LC_TIME, '')

'ru_RU.UTF-8'

In [6]:
client = MongoClient('localhost', 27017)
mongodb = client.mail
collection = mongodb['letters']

In [7]:
chrome_options = Options()
chrome_options.add_argument('start-maximized')

In [8]:
driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)

In [9]:
driver.get('https://mail.ru')

In [ ]:
try:
    WebDriverWait(driver, 60).until(EC.title_contains('Mail.ru'))
    try:
        login()
        WebDriverWait(driver, 60).until(EC.title_contains('Входящие - Почта Mail.ru'))
        actions = ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN)
        while True:
            actions.perform()
            time.sleep(0.5)
            try:
                elem = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'layout__letter-content')))
                try:
                    author = elem.find_element_by_class_name('letter__contact-item').text
                    date = get_date(elem)
                    theme = elem.find_element_by_class_name('thread__subject').text
                    text = get_text(elem)
                    collection.insert_one({'author' : author, 'date' : date, 'theme' : theme, 'text' : text})
                except:
                    print('something went wrong in getting data')
            except TimeoutException:
                break
    except:
        print('something went wrong in logining')
except:
    print('something went wrong in getting mail.ru')
driver.quit()